# Langchain integration
In a world where the shlux LLM we built is good enough for use, langchain proves to be a very convenient tool for interacting with the model. Langchain has allowed us to easily create a chatbot that can interact with the model. 

In [7]:
from langchain_core.language_models.llms import LLM
from langchain.callbacks.manager import CallbackManagerForLLMRun
from typing import Any, List, Optional
import sys

sys.path.append("../")
from gpt_model.model import GPT
from model_generate import text_to_tokens, generate, tokens_to_text
import tiktoken
import torch


class ShluxGPT(LLM):
    """
    Custom LLM class.
    """
    model: Any = None
    tokenizer: Any = None

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.model = GPT({
            "vocab_size": 50257,
            "n_embd": 768,
            "n_heads": 12,
            "n_layers": 12,
            "dropout": 0.1,
            "context_length": 256,
        })
        self.model.load_state_dict(torch.load("../pretraining_model/model.pth"))
        self.tokenizer = tiktoken.encoding_for_model("gpt2")

    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        token_ids = generate(
            model=self.model,
            idx=text_to_tokens(prompt, self.tokenizer),
            max_new_tokens=15,
            context_size=256,
            top_k=25,
            temperature=1.4,
        )
        return tokens_to_text(token_ids, self.tokenizer)

    @property
    def _llm_type(self) -> str:
        return "shluxgpt"

In [8]:
llm = ShluxGPT(n=5)
print(llm.invoke("Hello, how are you?"))


/tmp/ipykernel_5106/1954430276.py:30: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.load("../pretraining_model/model.pth"))


'Hello, how are you? A\nthat in the Party? It was almost impossible, it\ncould'